In [2]:
!pip install --upgrade pip
!pip install -U tiktoken -q
!pip install pytorch_lightning -q 
!pip install transformers -q 
!pip install torch -q 
!pip install datasets -q

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 120.6 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: python3 -m pip install --upgrade pip


In [90]:
from tqdm import tqdm

import pytorch_lightning as pl
import torch.nn as nn
import torch
import torch.nn.functional as F
import os, time
import glob
import tiktoken
from tqdm import tqdm
import json

from transformers import (
    AutoTokenizer, ElectraModel,
    ElectraForSequenceClassification, AutoModelForSequenceClassification,
    TrainingArguments, AutoModel,
    Trainer)

from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

import pandas as pd
import numpy as np

from openai import OpenAI

In [ ]:
# 디시인사이드의 사회적 논란 관련 텍스트에 대한 판별 및 회귀분석

# 1) 사전 파악된 사회적 논란 유형을 데이터분석의 형태에 맞게 재정렬
# 2) 휴먼 코딩을 학습
# 3) 수집된 게시물에 대한 분류 처리

In [4]:
# 1) CSV 로드 (인코딩은 utf-8-sig 권장)
df_meta = pd.read_csv('/home/work/CSG/snv/아티스트_사회적논란_변수통합_0704-1.csv', encoding="utf-8-sig")

# 2) month는 사건 설명용
df_meta["month_str"] = pd.to_datetime(  # month 컬럼을 날짜형으로 변환 후
    df_meta["month"], errors="coerce"   # 변환 불가 값은 NaT로 처리
).dt.strftime("%Y-%m")                  # "YYYY-MM" 형식의 문자열로 저장 (사건 시점 설명용)

# 3) 논란 카테고리 컬럼 범위 ("기타" ~ "특이사항")
cols = list(df_meta.columns)           # 전체 컬럼명을 리스트로 변환
start = cols.index("기타")             # "기타" 컬럼의 인덱스 위치
end   = cols.index("특이사항")         # "특이사항" 컬럼의 인덱스 위치
contro_cols = cols[start:end+1]        # "기타"부터 "특이사항"까지 구간에 해당하는 컬럼들만 추출 (논란 카테고리 컬럼 목록)

rows = []                               # 논란 케이스별 레코드를 쌓을 리스트

for row_idx, r in df_meta.iterrows():   # 메타 DataFrame의 각 행을 순회
    base_case_id = int(row_idx)         # 원본 행 인덱스를 사건 기본 ID로 사용

    for col in contro_cols:             # 각 논란 카테고리 컬럼에 대해 반복
        val = r[col]                    # 현재 행에서 해당 카테고리의 값
        if pd.isna(val):                # NaN이면 (값이 없으면) 건너뜀
            continue

        s = str(val).strip()            # 문자열로 변환 후 앞뒤 공백 제거
        # 0 또는 빈 문자열이면 논란 없음
        if s in ["", "0"]:              # 값이 공백이거나 "0"이면 논란이 없는 것으로 간주
            continue

        # "1 (설명…)" 형태 처리
        detail = None                   # 세부 설명 초기값
        if s.startswith("1"):           # 값이 "1" 또는 "1 (설명…)" 형태인 경우
            rest = s[1:].lstrip(" ()")  # 앞의 "1" 제거 후 남은 괄호/공백 제거
            detail = rest if rest else None  # 남은 문자열이 있으면 세부 설명으로 사용, 없으면 None
        elif s != "1":                  # 단순히 "1"이 아니라면
            detail = s                  # 전체 문자열을 세부 설명으로 사용

        # 하나의 논란 케이스 = (원본 행, 논란유형)
        case_id = f"{base_case_id}_{col}"  # 원본 행 인덱스와 컬럼명을 조합해 유니크한 케이스 ID 생성

        rows.append({                        # 한 논란 케이스를 딕셔너리로 만들어 rows에 추가
            "case_id": case_id,              # 케이스 ID
            "artist": r["artist"],           # 아티스트 이름
            "month": r["month_str"],         # 사건 시점(YYYY-MM 문자열, 설명용)
            "controversy_category_ko": col,  # 논란 유형 이름 (컬럼명 그대로 한글 카테고리)
            "controversy_detail_ko": detail, # 논란 세부 설명(있으면 문자열, 없으면 None)

            # 🔸 요청하신 5개 변수 유지
            "group": r.get("group"),         # 소속 그룹 정보
            "sex_target": r.get("sex_target"),  # 성별 관련 타겟 변수
            "acttype": r.get("acttype"),        # 활동 유형
            "listorg": r.get("listorg"),        # 리스트 / 기관 정보
            "type": r.get("type"),              # 사건 유형
        })

df_cases = pd.DataFrame(rows)            # 누적된 rows 리스트를 케이스 단위 DataFrame으로 변환

df_cases                                 # 결과 DataFrame 확인

,case_id,artist,month,controversy_category_ko,controversy_detail_ko,group,sex_target,acttype,listorg,type
0,0_태도 논란 (태도/발언/갑질),(여자)아이들 ((G)I-DLE),2023-06,태도 논란 (태도/발언/갑질),None,1,2,2,1,1
1,1_불법 프로그램 / 콘텐츠 사용,(여자)아이들 ((G)I-DLE),2019-03,불법 프로그램 / 콘텐츠 사용,불법 복제 소프트웨어 사용),1,2,2,1,1
2,2_지식재산권 (표절/저작권/상표권 등),(여자)아이들 ((G)I-DLE),2022-02,지식재산권 (표절/저작권/상표권 등),None,1,2,2,1,1
3,"3_SNS (게시물 콘텐츠, 댓글, 라이브 방송 등)",(여자)아이들 ((G)I-DLE),2016-12,"SNS (게시물 콘텐츠, 댓글, 라이브 방송 등)",None,1,2,2,1,1
4,"4_연애/이성 (스캔들, 열애설/공개연애/결혼)",(여자)아이들 ((G)I-DLE),2023-08,"연애/이성 (스캔들, 열애설/공개연애/결혼)",None,1,2,2,1,1
...,...,...,...,...,...,...,...,...,...,...
924,773_기타,헨리 (Henry),2021-10,기타,오성홍기 마스크 착용 논란,4,1,4,0,3
925,774_기타,헨리 (Henry),2021-10,기타,부채춤을 중국문화라고 소개했다는 루머(사실X),4,1,4,0,3
926,"775_SNS (게시물 콘텐츠, 댓글, 라이브 방송 등)",헨리 (Henry),2022-02,"SNS (게시물 콘텐츠, 댓글, 라이브 방송 등)",반중 유튜브 댓글 삭제 논란(사실X)),4,1,4,0,3
927,775_특이사항,헨리 (Henry),2022-02,특이사항,"한국 비하 댓글은 삭제하지 않으나, 중국을 비판하는 댓글만 골라서 삭제한다는 유언비어",4,1,4,0,3


In [135]:
len(set(df_cases['artist']))

152

In [74]:
# 1) 필요한 3개 컬럼만 추출
df_min = df_cases[["artist", "month", "controversy_category_ko"]].copy()  # 아티스트, 월, 논란 카테고리만 별도 DataFrame으로 복사

# (선택) 완전 동일한 행 있으면 제거
df_min = df_min.drop_duplicates()  # 같은 아티스트·같은 월·같은 카테고리가 완전히 중복이면 제거

# 2) artist 단위로 묶어서 JSON 구조 만들기
result = []  # 아티스트별 논란 이력을 담을 최종 리스트

for artist, g in df_min.groupby("artist"):  # 아티스트 이름을 기준으로 그룹화하여 반복
    # 각 아티스트에 대해 month + category 리스트 만들기
    cases = (                                                             
        g[["month", "controversy_category_ko"]]  # 현재 아티스트 그룹에서 월과 카테고리만 사용
        .rename(columns={"controversy_category_ko": "category"})  # JSON에서 보기 좋게 컬럼명 변경
        .to_dict(orient="records")  # [{"month": "...", "category": "..."}, ...] 형태의 리스트로 변환
    )
    result.append({                         # 아티스트 하나에 대한 구조를 딕셔너리로 추가
        "artist": artist,                   # 아티스트 이름
        "cases": cases                      # 해당 아티스트의 (월, 카테고리) 목록
    })

json_list = result  # 결과를 json_list 변수명으로도 보관 (추가 활용 용도)

# 4) 파일로 저장하고 싶으면
with open("/home/work/CSG/snv/controversy_by_artist.json", "w", encoding="utf-8") as f:  # JSON 파일을 UTF-8로 쓰기 모드 오픈
    json.dump(result, f, ensure_ascii=False, indent=4)  # 한글이 깨지지 않도록 ensure_ascii=False, 보기 좋게 들여쓰기해서 저장


In [75]:
json_list[0]

{'artist': '(여자)아이들 ((G)I-DLE)',
 'cases': [{'month': '2023-06', 'category': '태도 논란 (태도/발언/갑질)'},
  {'month': '2019-03', 'category': '불법 프로그램 / 콘텐츠 사용'},
  {'month': '2022-02', 'category': '지식재산권 (표절/저작권/상표권 등)'},
  {'month': '2016-12', 'category': 'SNS (게시물 콘텐츠, 댓글, 라이브 방송 등)'},
  {'month': '2023-08', 'category': '연애/이성 (스캔들, 열애설/공개연애/결혼)'},
  {'month': '2024-05', 'category': '태도 논란 (태도/발언/갑질)'},
  {'month': '2019-08', 'category': '정치 (활동/발언), 사회 이슈'},
  {'month': '2021-07', 'category': '정치 (활동/발언), 사회 이슈'},
  {'month': '2019-11', 'category': '인종차별 '},
  {'month': '2022-11', 'category': 'SNS (게시물 콘텐츠, 댓글, 라이브 방송 등)'},
  {'month': '2023-12', 'category': '가수 활동 (뮤직비디오, 무대, 노래가사, 의상, 퍼포먼스 등)'}]}

In [82]:
df_min = df_cases[["artist", "month", "controversy_category_ko"]].copy()  # 아티스트, 월, 논란 카테고리만 추출해서 별도 DataFrame 생성
df_min = df_min.drop_duplicates()                                         # 완전히 동일한 행(아티스트+월+카테고리)이면 중복 제거

# (artist, month)별 카테고리 문자열 만들기
artist_month_meta = (                                                     
    df_min
    .groupby(["artist", "month"], as_index=False)                          # 아티스트와 월 단위로 그룹화
    .agg(
        categories=(
            "controversy_category_ko",
            lambda x: ", ".join(sorted(set(x)))                            # 해당 (artist, month)에 속한 카테고리들을 중복 제거 후 정렬, "A, B, C" 형태 문자열로 결합
        )
    )
)

print(len(artist_month_meta))                                             # (artist, month) 조합의 총 개수 출력
print(len(set(artist_month_meta['artist'])))                              # 등장한 고유 아티스트 수 출력
artist_month_meta.head()                                                  # 결과 DataFrame의 상위 5행 확인


730
152


,artist,month,categories
0,(여자)아이들 ((G)I-DLE),2016-12,"SNS (게시물 콘텐츠, 댓글, 라이브 방송 등)"
1,(여자)아이들 ((G)I-DLE),2019-03,불법 프로그램 / 콘텐츠 사용
2,(여자)아이들 ((G)I-DLE),2019-08,"정치 (활동/발언), 사회 이슈"
3,(여자)아이들 ((G)I-DLE),2019-11,인종차별
4,(여자)아이들 ((G)I-DLE),2021-07,"정치 (활동/발언), 사회 이슈"


In [83]:
artist_month2cats = {                                           # (artist, month)를 키로, 카테고리 문자열을 값으로 가지는 딕셔너리 생성
    (row.artist, row.month): row.categories                     # 키: (아티스트 이름, 월), 값: "카테고리1, 카테고리2, ..." 형태 문자열
    for row in artist_month_meta.itertuples(index=False)        # artist_month_meta의 각 행을 튜플 형태로 순회 (index는 사용하지 않음)
}

artist_month2cats                                               # 생성된 매핑 딕셔너리 내용 확인

{('(여자)아이들 ((G)I-DLE)', '2016-12'): 'SNS (게시물 콘텐츠, 댓글, 라이브 방송 등)',
 ('(여자)아이들 ((G)I-DLE)', '2019-03'): '불법 프로그램 / 콘텐츠 사용',
 ('(여자)아이들 ((G)I-DLE)', '2019-08'): '정치 (활동/발언), 사회 이슈',
 ('(여자)아이들 ((G)I-DLE)', '2019-11'): '인종차별 ',
 ('(여자)아이들 ((G)I-DLE)', '2021-07'): '정치 (활동/발언), 사회 이슈',
 ('(여자)아이들 ((G)I-DLE)', '2022-02'): '지식재산권 (표절/저작권/상표권 등)',
 ('(여자)아이들 ((G)I-DLE)', '2022-11'): 'SNS (게시물 콘텐츠, 댓글, 라이브 방송 등)',
 ('(여자)아이들 ((G)I-DLE)', '2023-06'): '태도 논란 (태도/발언/갑질)',
 ('(여자)아이들 ((G)I-DLE)', '2023-08'): '연애/이성 (스캔들, 열애설/공개연애/결혼)',
 ('(여자)아이들 ((G)I-DLE)', '2023-12'): '가수 활동 (뮤직비디오, 무대, 노래가사, 의상, 퍼포먼스 등)',
 ('(여자)아이들 ((G)I-DLE)', '2024-05'): '태도 논란 (태도/발언/갑질)',
 ('2NE1', '2009-08'): '가수 활동 (뮤직비디오, 무대, 노래가사, 의상, 퍼포먼스 등)',
 ('2NE1', '2010-10'): '마약',
 ('2NE1', '2014-03'): '가수 활동 (뮤직비디오, 무대, 노래가사, 의상, 퍼포먼스 등)',
 ('2NE1', '2015-08'): '인종차별 ',
 ('2NE1', '2016-04'): '정치 (활동/발언), 사회 이슈',
 ('2NE1', '2017-01'): 'SNS (게시물 콘텐츠, 댓글, 라이브 방송 등)',
 ('2NE1', '2018-04'): '기타',
 ('2am', '2013-08'): '태도 논란 (태도/발언/

* 휴먼 코딩 데이터 입력

In [6]:
df_human = pd.read_csv('/home/work/CSG/snv/사회적논란-휴먼코딩.csv', encoding='utf-8')
df_human['texts'] = df_human['title'] + ' ' + df_human['content']

df_human

,artist,month,title,content,논란여부,논란유형,date,ip,like,nickname,...,reconsump_cn,ureconsump_cn,wuemr,wemr,emprate,new_dc_post,new_dc_view,new_dc_recom,new_dc_uid,texts
0,2NE1,2025-10-10,공지. 친목질 하지마,특히 교무실 드립 어쩌구 하는애들 알았지? 어서 친목질이야 ㅋㅋㅋ,0,마약,2010-10-12 0:00,NaN,0.0,친목질하지마,...,10845.0,17119.0,6.29,58.64,59.5,4887.0,1883248.0,410.0,3240.0,공지. 친목질 하지마 특히 교무실 드립 어쩌구 하는애들 알았지? 어서 친목질이야 ㅋㅋㅋ
1,2NE1,2025-10-10,조공북..희망사항..,희망사항은 백만개지만...짤로 만드는건 어렵긔....ㅋ,0,마약,2010-10-12 0:00,NaN,0.0,하늘벽tj,...,10845.0,17119.0,6.29,58.64,59.5,4887.0,1883248.0,410.0,3240.0,조공북..희망사항.. 희망사항은 백만개지만...짤로 만드는건 어렵긔....ㅋ
2,B.A.P,NaN,멋찬 와꾸도 젤루한테 발리고 무쓸모 맞는것같은데,얼빠급식들 빨던게 젤루잖어,0,성범죄,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,멋찬 와꾸도 젤루한테 발리고 무쓸모 맞는것같은데 얼빠급식들 빨던게 젤루잖어
3,EXO-K,2025-06-14,어른유글은 다 신고삭제잼 ㅋㅋㅋㅋ,그래봐야 어른유가 남자들한테 아닌척 꼬리치는 상여우인거는 유명함,1,스캔들,2014-06-19 0:00,223.620,5.0,ㅇㅇ,...,17680.0,25264.0,5.97,58.11,61.2,7710.0,2574410.0,5403.0,2838.0,어른유글은 다 신고삭제잼 ㅋㅋㅋㅋ 그래봐야 어른유가 남자들한테 아닌척 꼬리치는 상...
4,EXO-K,2025-06-14,이때까지 괜찮았는데 스벅갔다는 글보고,진짜 피꺼솟ㅋㅋㅋㅋㅋㅋ 돌았나 저게,0,스캔들,2014-06-19 0:00,223.620,3.0,ㅇㅇ,...,17680.0,25264.0,5.97,58.11,61.2,7710.0,2574410.0,5403.0,2838.0,이때까지 괜찮았는데 스벅갔다는 글보고 진짜 피꺼솟ㅋㅋㅋㅋㅋㅋ 돌았나 저게
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1016,티파니 (TIFFANY),2025-04-14,오랜만에 로그인도 하고 팊갤도 왔는데 생각한것 보단 병신들이 별로없네,거의 야갤에 있어서 그런가 다들 힘들겠지만 어떡하겠어 이미 벌어진건데 갤러들 다들 ...,1,스캔들,2014-04-05 0:00,NaN,0.0,女묭묭이神,...,17680.0,25264.0,5.97,58.11,60.9,6866.0,1385407.0,8813.0,4199.0,오랜만에 로그인도 하고 팊갤도 왔는데 생각한것 보단 병신들이 별로없네 거의 야갤에 ...
1017,티파니 (TIFFANY),2025-04-14,여기 갤분위기 진짜 아련아련하다..,타팬인데 소시도 좋아해서 가끔 눈팅하는데 여기는 티파니가 눈팅도 해서 그런지 조심조...,1,스캔들,2014-04-04 0:00,175.223,5.0,ㅇㅇ,...,17680.0,25264.0,5.97,58.11,60.9,6866.0,1385407.0,8813.0,4199.0,여기 갤분위기 진짜 아련아련하다.. 타팬인데 소시도 좋아해서 가끔 눈팅하는데 여기는...
1018,티파니 (TIFFANY),2025-04-14,이런 짤도 좋아여,NaN,0,스캔들,2014-04-04 0:00,NaN,0.0,묭타민!_!,...,17680.0,25264.0,5.97,58.11,60.9,6866.0,1385407.0,8813.0,4199.0,NaN
1019,티파니 (TIFFANY),2025-04-14,파니야 지금 어때..,난 지금 되게 뭔가 힘들다...ㅎㅎ.... 내가 지금 뭘해야할지 도통 제대로 감잡...,0,스캔들,2014-04-04 0:00,NaN,1.0,첼셜레,...,17680.0,25264.0,5.97,58.11,60.9,6866.0,1385407.0,8813.0,4199.0,파니야 지금 어때.. 난 지금 되게 뭔가 힘들다...ㅎㅎ.... 내가 지금 뭘해야...


In [140]:
set(df_human['논란유형'])

{'마약', '성범죄', '스캔들', '태도논란'}

* kcelectra 기반 분류기 적용

In [85]:
# 1) 기본 컬럼 가져오기
df_human1 = df_human[["artist", "month", "texts", "논란여부"]].copy()  # 원본 df_human에서 필요한 컬럼만 따로 복사하여 작업용 DataFrame 생성

# 2) 결측 및 타입 방어
df_human1["texts"]  = df_human1["texts"].fillna("").astype(str)        # texts 컬럼의 결측치를 빈 문자열로 채우고, 전부 문자열 타입으로 변환
df_human1["artist"] = df_human1["artist"].fillna("UNKNOWN").astype(str)  # artist 컬럼의 결측치는 "UNKNOWN"으로 채우고 문자열로 변환
df_human1["month"]  = df_human1["month"].astype(str)                   # month 컬럼도 이후 매핑 편의를 위해 문자열로 변환

# 3) 아티스트 전체 논란 메타 (artist_meta_simple에서 만든 dict 사용)
df_human1["has_controversy_artist"] = df_human1["artist"].map(artist2has_contro).fillna(0).astype(int)  # 아티스트별 논란 존재 여부를 dict에서 가져와 매핑 (없으면 0)
df_human1["n_cases_artist"]         = df_human1["artist"].map(artist2ncases).fillna(0).astype(int)      # 아티스트별 논란 건수 정보를 dict에서 가져와 매핑 (없으면 0)

# 4) (artist, month)별 카테고리 문자열 매핑
def get_month_cats(row):                                              # 한 행(row)에 대해 (artist, month)에 해당하는 카테고리 문자열을 찾아주는 함수
    return artist_month2cats.get((row["artist"], row["month"]), "")   # (artist, month)를 키로 딕셔너리 조회, 없으면 빈 문자열 반환

df_human1["month_categories"] = df_human1.apply(get_month_cats, axis=1)  # 각 행에 대해 월별 논란 카테고리 문자열을 계산해 새로운 컬럼으로 추가

# 5) 최종 input_text 만들기
def make_input_text(row):                                             # LLM에 넣을 최종 입력 텍스트를 한 줄로 구성하는 함수
    artist = row["artist"]                                            # 아티스트 이름
    month  = row["month"]                                             # 월 정보 (문자열)
    base   = row["texts"]                                             # 원본 텍스트(사람이 쓴 코멘트 등)

    cats   = row["month_categories"]       # 해당 (artist, month)에 해당하는 논란 카테고리들 (예: "정치, 사회 이슈")
    has_cats = 1 if cats != "" else 0      # 해당 월에 논란 카테고리가 있으면 1, 없으면 0

    meta_tag = (                          # 메타 정보를 태그 형식으로 하나의 문자열로 구성
        f"[ARTIST={artist}] "             # 아티스트 이름 태그
        f"[MONTH={month}] "               # 월 정보 태그
        f"[HAS_CONTROV_ARTIST={row['has_controversy_artist']}] "  # 이 아티스트에게 전체 기간 기준 논란이 있었는지 여부
        f"[N_CASES_ARTIST={row['n_cases_artist']}] "              # 이 아티스트의 전체 논란 건수
        f"[HAS_CONTROV_MONTH={has_cats}] "                        # 이 달(month)에 논란이 있었는지 여부
        f"[CATS_MONTH={cats}]"                                    # 이 달에 해당하는 논란 카테고리 묶음
    )
    return meta_tag + " " + base  # 메타 태그와 실제 원문 텍스트를 공백 하나로 이어 붙여 최종 input 텍스트로 사용

df_human1["input_text"] = df_human1.apply(make_input_text, axis=1)    # 각 행에 대해 make_input_text를 적용하여 input_text 컬럼 생성
df_human1["label"]      = df_human1["논란여부"].astype(int)           # 사람이 라벨링한 논란 여부(0/1)를 정수형으로 변환하여 label 컬럼에 저장

In [86]:
train_df = df_human1[["input_text", "label"]].reset_index(drop=True).copy()  # 모델 학습에 필요한 컬럼만(input_text, label) 선택하고 인덱스를 0부터 다시 부여한 뒤 복사본 생성
train_ds = Dataset.from_pandas(train_df)                                     # pandas DataFrame을 Hugging Face Dataset 형식으로 변환

raw_datasets = DatasetDict({                                                 # 여러 개의 데이터셋을 하나의 딕셔너리처럼 묶어 관리하기 위한 DatasetDict 생성
    "train": train_ds                                                        # "train" 키 아래에 방금 만든 학습용 Dataset을 넣음
})

print(raw_datasets)                                                          # 전체 DatasetDict 구조와 크기를 출력하여 확인

DatasetDict({
    train: Dataset({
        features: ['input_text', 'label'],
        num_rows: 1021
    })
})


In [91]:
MODEL_NAME = "beomi/KcELECTRA-base"  # 사용할 사전학습 모델 이름 지정 (한국어용 KcELECTRA base 모델)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)  # 지정한 모델과 맞는 토크나이저 로드 (문장을 토큰 ID로 바꿔주는 역할)

model = AutoModelForSequenceClassification.from_pretrained(  # 사전학습된 모델 위에 분류용 헤드를 올린 형태로 로드
    MODEL_NAME,                                              # 백본(backbone)으로 사용할 사전학습 모델 이름
    num_labels=2  # 0/1 이진 분류                           # 출력 차원을 2로 설정 (예: 0=논란 아님, 1=논란 있음)
)

device = "cuda" if torch.cuda.is_available() else "cpu"  # GPU 사용 가능하면 'cuda', 아니면 'cpu' 선택
model.to(device)  # 선택한 디바이스로 모델 이동 (이후 학습/추론 시 같은 디바이스를 사용해야 함)

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at beomi/KcELECTRA-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(30000, 768, padding_idx=3)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0-11): 12 x ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): L

In [92]:
MAX_LEN = 256  # 필요에 따라 조정  # 한 문장을 토큰으로 자를 때 최대 길이(토큰 수). 너무 길면 자르고, 짧으면 패딩으로 채움

def tokenize_fn(batch):  # 데이터셋의 배치를 받아 토크나이즈하는 함수 정의
    return tokenizer(
        batch["input_text"],          # 우리가 이전에 만든 input_text 컬럼(메타태그 + 원문 텍스트)
        padding="max_length",         # 길이가 MAX_LEN보다 짧으면 뒤를 패딩 토큰으로 채움
        truncation=True,              # 길이가 MAX_LEN을 넘으면 잘라냄
        max_length=MAX_LEN            # 최대 길이 설정
    )

tokenized_datasets = raw_datasets.map(   # HuggingFace Dataset에 토크나이즈 함수를 적용해서 새로운 토큰화된 데이터셋 생성
    tokenize_fn,                         # 위에서 정의한 tokenize_fn 사용
    batched=True,                        # 한 번에 여러 개의 샘플을 묶어서 처리 (속도↑)
    remove_columns=["input_text"]        # 토크나이즈 후에는 원본 문자열 컬럼은 제거 (이미 토큰 ID로 변환되었기 때문)
)

tokenized_datasets.set_format(           # Dataset에서 어떤 형식으로 데이터를 꺼낼지 지정
    type="torch",                        # PyTorch 텐서 형식으로 반환하도록 설정
    columns=["input_ids", "attention_mask", "label"]  # 모델에 필요로 하는 컬럼만 남김 (입력 ID, 마스크, 라벨)
)

tokenized_datasets  # 토큰화된 전체 데이터셋 구조를 확인 (train 등 split과 각 길이/컬럼 정보)

Map:   0%|          | 0/1021 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1021
    })
})

In [93]:
training_args = TrainingArguments(                          # 학습에 사용할 각종 하이퍼파라미터와 설정을 담는 객체
    output_dir="./kcelectra_controversy_meta",             # 학습된 모델과 체크포인트를 저장할 디렉터리 경로
    learning_rate=5e-5,                                   # 옵티마이저에 사용할 학습률 (너무 크면 발산, 너무 작으면 느리게 학습)
    per_device_train_batch_size=32,                       # GPU/CPU 한 개당 한 번에 넣을 배치 크기 (메모리와 속도에 직접 영향)
    num_train_epochs=3,                                   # 전체 학습 데이터셋을 몇 번 반복해서 볼지(에폭 수)
    weight_decay=0.01,                                    # 가중치 감쇠(L2 정규화) 계수, 과적합 방지 역할
    logging_steps=100,                                    # 몇 스텝마다 학습 로그(손실 등)를 출력할지
    logging_dir="./logs",                                 # 로그 파일을 저장할 디렉터리 (TensorBoard 등에서 사용 가능)
)

trainer = Trainer(                                         # HuggingFace의 Trainer 객체 생성 (학습 루프를 자동으로 관리해주는 클래스)
    model=model,                                          # 앞에서 준비한 분류 모델 (KcELECTRA + 분류 헤드)
    args=training_args,                                   # 위에서 정의한 학습 설정들
    train_dataset=tokenized_datasets["train"],            # 학습에 사용할 토큰화된 Dataset (input_ids, attention_mask, label 포함)
    tokenizer=tokenizer,                                  # 토큰화 정보(패딩/마스크 등)를 위해 토크나이저도 함께 전달
)

trainer.train()                                            # 학습 시작. 에폭 수만큼 데이터 반복하며 손실 감소 방향으로 파라미터 업데이트


/tmp/ipykernel_121/2867686957.py:11: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss


TrainOutput(global_step=96, training_loss=0.5159726540247599, metrics={'train_runtime': 981.3305, 'train_samples_per_second': 3.121, 'train_steps_per_second': 0.098, 'total_flos': 402954581283840.0, 'train_loss': 0.5159726540247599, 'epoch': 3.0})

In [94]:
save_dir = "./kcelectra_controversy_meta_final"
trainer.save_model(save_dir)
tokenizer.save_pretrained(save_dir)

('./kcelectra_controversy_meta_final/tokenizer_config.json',
 './kcelectra_controversy_meta_final/special_tokens_map.json',
 './kcelectra_controversy_meta_final/tokenizer.json')

In [95]:
df = pd.read_csv('/home/work/CSG/snv/df_final.csv', encoding='utf-8')
df['texts'] = df['title'] + ' ' + df['content']

print(len(df))
df.head()

/tmp/ipykernel_121/1269365606.py:1: DtypeWarning: Columns (2,3,8,10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/home/work/CSG/snv/df_final.csv', encoding='utf-8')


853129


,artist,content,ip,like,month,nickname,title,uid,unlike,url,view,texts
0,FT아일랜드 (FTISLAND),흠좆무네 ㅋㅋㅋ,203.227,0,2009-10,애기미나리,니들 레알 열두시 땡하면 나올거임?,NaN,0,https://gall.dcinside.com/board/view/?id=ftisl...,23,니들 레알 열두시 땡하면 나올거임? 흠좆무네 ㅋㅋㅋ
1,FT아일랜드 (FTISLAND),새벽반 횽들 소환!!,203.227,0,2009-10,애기미나리,새벽반 횽들 소환!!!!!!!!!!!,NaN,0,https://gall.dcinside.com/board/view/?id=ftisl...,29,새벽반 횽들 소환!!!!!!!!!!! 새벽반 횽들 소환!!
2,FT아일랜드 (FTISLAND),기타솔로 좋지 않음?,203.227,1,2009-10,애기미나리,어쩌란 말이야 ㅠㅠㅠㅠ (소리주의),NaN,0,https://gall.dcinside.com/board/view/?id=ftisl...,41,어쩌란 말이야 ㅠㅠㅠㅠ (소리주의) 기타솔로 좋지 않음?
3,FT아일랜드 (FTISLAND),학원갔다 왔더니 정전쩜니다ㅠㅠ,NaN,0,2009-10,정용화,개럴들 다 어디감 ㅠㅠ,slove92,0,https://gall.dcinside.com/board/view/?id=ftisl...,34,개럴들 다 어디감 ㅠㅠ 학원갔다 왔더니 정전쩜니다ㅠㅠ
4,FT아일랜드 (FTISLAND),성인의 남성미가 물씬 풍기네여,203.227,0,2009-10,애기미나리,뮤뱅 애기 짤,NaN,0,https://gall.dcinside.com/board/view/?id=ftisl...,41,뮤뱅 애기 짤 성인의 남성미가 물씬 풍기네여


In [127]:
sample_n = 20000  # 샘플링할 데이터 개수 설정 (예: 20,000건만 추려서 예측 실험)
df_sample = df.sample(n=sample_n, random_state=42).copy()  # 전체 df에서 무작위로 sample_n개 추출 (random_state로 재현성 보장)
df_sample.reset_index(drop=True, inplace=True)  # 기존 인덱스는 버리고 0부터 다시 인덱스를 붙임

df_sample_pred = df_sample.copy()  # 예측용 전처리를 위해 별도의 복사본 생성

df_sample_pred["texts"]  = df_sample_pred["texts"].fillna("").astype(str)        # texts 컬럼의 결측치는 빈 문자열로 채우고 문자열 타입으로 통일
df_sample_pred["artist"] = df_sample_pred["artist"].fillna("UNKNOWN").astype(str)  # artist 결측치는 "UNKNOWN"으로 채우고 문자열로 변환
df_sample_pred["month"]  = df_sample_pred["month"].astype(str)                  # month 컬럼도 문자열로 변환 (딕셔너리 키로 활용하기 위함)

# 아티스트 전체 논란 메타
df_sample_pred["has_controversy_artist"] = df_sample_pred["artist"].map(artist2has_contro).fillna(0).astype(int)  # 아티스트별 논란 여부(있으면 1, 없으면 0)를 매핑
df_sample_pred["n_cases_artist"]         = df_sample_pred["artist"].map(artist2ncases).fillna(0).astype(int)      # 아티스트별 논란 건수(0 이상 정수)를 매핑

# (artist, month)별 카테고리 문자열
def get_month_cats(row):  # 한 행(row)에 대해 해당 아티스트의 특정 month에 어떤 논란 카테고리가 있었는지 찾아주는 함수
    return artist_month2cats.get((row["artist"], row["month"]), "")  # (artist, month)를 키로 딕셔너리 조회, 없으면 빈 문자열 반환

df_sample_pred["month_categories"] = df_sample_pred.apply(get_month_cats, axis=1)  # 각 행에 대해 월별 논란 카테고리 문자열 계산 후 컬럼으로 추가

def make_input_text_for_df(row):  # LLM/분류 모델에 넣을 최종 input_text를 만드는 함수 (메타태그 + 원문 텍스트)
    artist = row["artist"]  # 현재 행의 아티스트 이름
    month  = row["month"]   # 현재 행의 월 정보 (문자열)
    base   = row["texts"]   # 현재 행의 원문 텍스트

    cats   = row["month_categories"]          # 해당 아티스트-월에 대한 논란 카테고리 문자열 (예: "정치, 사회 이슈")
    has_cats = 1 if cats != "" else 0         # 카테고리가 하나라도 있으면 1, 없으면 0

    meta_tag = (                              # 각종 메타 정보를 태그 형태로 하나의 문자열로 묶음
        f"[ARTIST={artist}] "                 # 아티스트 이름 태그
        f"[MONTH={month}] "                   # 월 정보 태그
        f"[HAS_CONTROV_ARTIST={row['has_controversy_artist']}] "  # 이 아티스트에게 과거 논란이 있었는지 여부
        f"[N_CASES_ARTIST={row['n_cases_artist']}] "              # 이 아티스트의 전체 논란 건수
        f"[HAS_CONTROV_MONTH={has_cats}] "                        # 이 아티스트가 해당 월에 논란이 있었는지 여부
        f"[CATS_MONTH={cats}]"                                    # 해당 월의 논란 카테고리들
    )
    return meta_tag + " " + base  # 메타태그와 실제 텍스트를 공백 한 칸으로 이어 붙여 최종 input 문자열 생성

df_sample_pred["input_text"] = df_sample_pred.apply(make_input_text_for_df, axis=1)  # 모든 행에 대해 input_text 생성 함수 적용

In [128]:
model.eval()                              # 모델을 평가 모드로 전환 (dropout, batchnorm 등을 비활성화하여 추론에 적합한 상태로 만듦)
model.to(device)                          # 모델을 GPU 또는 CPU로 이동 (앞에서 설정한 device 변수 사용)

def tokenize_for_inference(batch_texts):  # 여러 개의 문장을 한 번에 토크나이즈하기 위한 함수 정의
    return tokenizer(
        batch_texts,                      # 리스트 형태의 문장들 (["문장1", "문장2", ...])
        padding=True,                     # 배치 내 가장 긴 문장 길이에 맞춰 나머지를 패딩
        truncation=True,                  # 최대 길이를 넘는 문장은 잘라냄
        max_length=MAX_LEN,               # 앞에서 정의한 MAX_LEN (예: 256 토큰)
        return_tensors="pt"               # PyTorch 텐서 형태로 반환 (모델에 바로 넣기 위함)
    )

all_texts = df_sample_pred["input_text"].tolist()  # 예측에 사용할 input_text 컬럼을 파이썬 리스트로 변환
batch_size = 64                                    # 한 번에 처리할 배치 크기 (GPU 메모리 상황에 따라 조정 가능)

all_preds = []                                     # 각 샘플에 대한 최종 0/1 예측값을 저장할 리스트
all_probs = []                                     # 각 샘플에 대해 "논란 있음(클래스 1)"의 확률을 저장할 리스트

with torch.no_grad():                              # 추론 모드: 기울기 계산 비활성화 (메모리 절약 및 속도 향상)
    num_batches = (len(all_texts) + batch_size - 1) // batch_size  # 전체 배치를 몇 번으로 나눌지 계산
    for i in tqdm(range(0, len(all_texts), batch_size), total=num_batches):  # 0부터 전체 길이까지 batch_size씩 증가하며 반복, 진행 상황은 tqdm으로 표시
        batch_texts = all_texts[i:i+batch_size]   # 이번 배치에서 처리할 텍스트 슬라이스 추출
        enc = tokenize_for_inference(batch_texts) # 배치 단위 텍스트를 토크나이즈하여 인코딩(입력 ID, 마스크 등) 생성
        enc = {k: v.to(device) for k, v in enc.items()}  # 인코딩된 텐서들을 모두 GPU/CPU 디바이스로 이동

        outputs = model(**enc)                    # 모델에 인코딩된 입력을 전달하여 로짓(logits) 출력 얻기
        logits = outputs.logits                   # (배치크기, 2) 형태의 로짓 텐서 (각 클래스의 점수)

        preds  = torch.argmax(logits, dim=-1).cpu().numpy()    # 각 샘플에서 가장 큰 점수를 가진 클래스 인덱스 선택 → 0 또는 1로 예측
        probs  = F.softmax(logits, dim=-1)[:, 1].cpu().numpy() # softmax로 각 클래스의 확률 계산 후, 클래스 1(논란 있음)의 확률만 추출

        all_preds.extend(list(preds))             # 현재 배치의 예측 결과(0/1)를 전체 리스트에 이어 붙이기
        all_probs.extend(list(probs))             # 현재 배치의 클래스 1 확률을 전체 리스트에 이어 붙이기

df_sample_pred["논란여부_KcELECTRA"] = all_preds      # 예측된 0/1 값을 새로운 컬럼으로 DataFrame에 저장
df_sample_pred["논란확률_KcELECTRA"] = all_probs      # 예측된 확률 값을 또 다른 컬럼으로 저장 (모델의 확신 정도 확인용)


100% 313/313 [15:44<00:00,  3.02s/it]


In [129]:
df_sample_pred.head()

,artist,content,ip,like,month,nickname,title,uid,unlike,url,view,texts,has_controversy_artist,n_cases_artist,month_categories,input_text,논란여부_KcELECTRA,논란확률_KcELECTRA
0,JYJ,도키도키!!!!!!!! 내일이면 움직이는 티저가.........!!!!!!!,211.36,0,2015-02,레오뿌냐,현재 이 시각 꽃 발매와 꽃콘까지 남은 시간...!,NaN,0.0,https://gall.dcinside.com/board/view/?id=jyj&n...,220.0,현재 이 시각 꽃 발매와 꽃콘까지 남은 시간...! 도키도키!!!!!!!! 내일이...,1,21,"성범죄 (성희롱, 성추행, 성폭력, 불법촬영 등)",[ARTIST=JYJ] [MONTH=2015-02] [HAS_CONTROV_ARTI...,0,0.097260
1,마마무 (Mamamoo),[1위 예측] 옥탑방 (-0.09285) 4.82998|5.23084|4.9508...,NaN,0,2019-03,맘갤차트봇,[14:15] 멜론 5분 차트,mgchartbot2,0.0,https://gall.dcinside.com/board/view/?id=mamam...,59,[14:15] 멜론 5분 차트 [1위 예측] 옥탑방 (-0.09285) 4.829...,1,10,지식재산권 (표절/저작권/상표권 등),[ARTIST=마마무 (Mamamoo)] [MONTH=2019-03] [HAS_CO...,0,0.092829
2,엑소 (EXO),ㅋ,175.223,3,2017-12,ㅇㅇ,뿡 머상주기 싫어서 가온이 기사냄ㅋ,NaN,0,https://gall.dcinside.com/board/view/?id=exo_n...,426,뿡 머상주기 싫어서 가온이 기사냄ㅋ ㅋ,1,37,"태도 논란 (태도/발언/갑질), 특이사항",[ARTIST=엑소 (EXO)] [MONTH=2017-12] [HAS_CONTROV...,0,0.135178
3,엑소 (EXO),ㅈㄴ 싫어,210.104,0,2019-01,ㅇㅇ,혜나 왜저러냐 진짜,NaN,0,https://gall.dcinside.com/board/view/?id=exo_n...,20,혜나 왜저러냐 진짜 ㅈㄴ 싫어,1,37,"연애/이성 (스캔들, 열애설/공개연애/결혼), 지식재산권 (표절/저작권/상표권 등)...",[ARTIST=엑소 (EXO)] [MONTH=2019-01] [HAS_CONTROV...,0,0.341638
4,마마무 (Mamamoo),에어팟 1vs2,118.40,0,2019-03,ㅇㅇ,ㅃ vs,NaN,0,https://gall.dcinside.com/board/view/?id=mamam...,130,ㅃ vs 에어팟 1vs2,1,10,지식재산권 (표절/저작권/상표권 등),[ARTIST=마마무 (Mamamoo)] [MONTH=2019-03] [HAS_CO...,0,0.095476


In [130]:
df_sample_pred['논란여부_KcELECTRA'].value_counts()

논란여부_KcELECTRA
0    18098
1     1902
Name: count, dtype: int64

In [131]:
df_sample_pred.to_csv(
    "/home/work/CSG/snv/df_sample_with_kcelectra.csv",
    index=False,
    encoding="utf-8-sig"
)

# openAI

In [102]:
# OpenAI 파이썬 라이브러리에서 OpenAI 클래스 불러오기

api_key = ""

client = OpenAI(  # OpenAI API에 요청을 보내기 위한 클라이언트 객체 생성
  api_key= api_key
)

response = client.responses.create(  # OpenAI 모델에 요청을 보내고 응답(response) 객체 생성
  model="gpt-5-mini-2025-08-07",  # 사용할 모델 이름 지정
  input="write a haiku about ai"  # 모델에게 줄 프롬프트(명령/질문) 텍스트
)

print(response.output_text);  # 응답 객체에서 생성된 텍스트만 꺼내어 출력

Silent circuits dream
Learning in electric night
New dawn finds its voice


In [164]:
SYS_PROMPT = (
    "당신은 K-pop 아이돌 및 연예인 관련 온라인 게시물을 분류하는 라벨러입니다.\n"
    "주어진 글이 '사회적 논란'과 관련된 글인지 판별하십시오.\n\n"
    "라벨 정의는 다음과 같습니다.\n"
    "0: 사회적 논란과 관련이 없는 일반 글\n"
    "   - 예: 일상 팬질, 컴백/활동 정보, 성과 자랑, 잡담, 밈/농담 등\n\n"
    "1: 특정 아이돌/연예인의 사회적 논란과 관련된 글\n"
    "   - 예: 학교폭력, 음주운전, 마약, 범죄, 갑질, 혐오 발언, 정치·외교 논란,\n"
    "         병역/탈세 문제, 팬 기망, 표절/저작권 논란, 불법 행위 등\n"
    "   - 논란 사건 자체에 대한 뉴스/루머/후속 여파/옹호·비판·정보 공유 등 포함\n\n"
    "   - 해당 키워드가 직접 나타나지 않더라도 조금이라도 관련성 있어보이는 글, 아티스트에 대한 비난도 포함\n\n"
    "2: 텍스트만으로 0/1을 명확히 판단하기 어렵거나, 논란 관련성 여부가 애매한 글\n"
    "   - 예: 맥락이 너무 짧거나 모호해서 논란 관련인지 알 수 없음\n"
    "   - 예: 논란을 연상할 수 있지만 명시적으로 드러나지 않아 판단이 불확실함\n\n"
    "반드시 숫자 하나(0, 1, 2 중 하나)만 출력하십시오.\n"
    "다른 설명, 문장, 마크다운, 접두어, 공백 없이 숫자만 출력해야 합니다."
)

In [ ]:
df_sample_pred = df_sample_pred.copy()  # 원본에 영향이 가지 않도록 복사본을 만들어 작업
df_sample_pred["llm_contro_label"] = None  # LLM이 예측한 논란 여부(0/1/2)를 저장할 컬럼 초기화
df_sample_pred["llm_contro_raw"]   = None  # LLM의 원문 응답 전체를 저장할 컬럼 초기화

for idx, row in tqdm(df_sample_pred.iterrows(), total=len(df_sample_pred)):  # DataFrame의 각 행을 진행바(tqdm)와 함께 순회
    text   = str(row.get("texts", ""))                 # 본문 텍스트 (결측 시 빈 문자열)
    artist = str(row.get("artist", "알 수 없음"))      # 아티스트 이름 (결측 시 "알 수 없음")
    month  = str(row.get("month", "알 수 없음"))       # 게시물 월 정보 (YYYY-MM, 결측 시 "알 수 없음")

    # 메타데이터 (없으면 0/빈 문자열로 대체)
    has_contr_artist = int(row.get("has_controversy_artist", 0))  # 해당 아티스트에게 과거 논란 이력이 있는지(0/1)
    n_cases_artist   = int(row.get("n_cases_artist", 0))          # 해당 아티스트의 전체 논란 건수
    month_cats       = str(row.get("month_categories", ""))       # 해당 월(또는 인근 시기)의 논란 카테고리 문자열

    USER_PROMPT = f"""다음은 DC인사이드 등 온라인 커뮤니티에 올라온 게시물입니다.
이 글이 특정 아이돌/연예인의 '사회적 논란'과 관련된 글인지 분류하십시오.

[메타데이터]
- 아티스트: {artist}
- 게시물 월: {month}
- 이 아티스트 전체 논란 건수(메타데이터 기준): {n_cases_artist}
- 이 아티스트가 과거 논란 이력이 있는가(0/1): {has_contr_artist}
- 이 아티스트의 해당 월(및 인근 시기) 논란 카테고리(있으면): {month_cats if month_cats else "없음"}

[게시물 내용]
{text}

분류 규칙:
- 0: 사회적 논란과 관련 없는 일반 글
    (팬질, 활동 정보, 잡담, 드립 등)
- 1: 사회적 논란(학폭, 음주운전, 막말, 혐오 발언, 병역/탈세, 범죄, 정치·외교 논란 등)과 관련된 것으로 추정되는 글
    (논란 사건 자체, 후폭풍, 옹호/비판, 정보 공유 등 포함)
- 2: 텍스트만으로 논란 관련 여부를 명확히 판단하기 어려운 애매한 글

출력 형식:
- 0, 1, 2 중 하나만 숫자로 출력
"""  # LLM에게 전달할 사용자 프롬프트: 메타데이터 + 게시물 내용 + 라벨 정의 + 출력 형식 설명

    # 예외 / 레이트 리밋 대비: 최대 3번까지 재시도
    last_error = None  # 마지막 에러 메시지를 저장하기 위한 변수
    for attempt in range(3):  # 최대 3회까지 재시도
        try:
            resp = client.chat.completions.create(              # OpenAI Chat Completion API 호출
                model="gpt-5-mini-2025-08-07",                  # 사용할 LLM 모델 이름
                max_completion_tokens=4,                        # 숫자 1글자 정도만 필요하므로 토큰 수를 아주 작게 제한
                messages=[
                    {"role": "system", "content": SYS_PROMPT},  # 시스템 프롬프트: 모델의 역할/스타일 정의
                    {"role": "user",   "content": USER_PROMPT}, # 유저 프롬프트: 실제 분류 작업 지시 및 입력 텍스트
                ],
            )
            break  # 호출이 성공하면 재시도 루프 탈출
        except Exception as e:  # 네트워크 오류, 레이트 리밋 등 예외 처리
            last_error = e
            print(f"[ERROR] idx={idx}, attempt={attempt+1}, error={e}")  # 어떤 인덱스에서 몇 번째 시도에 실패했는지 로그 출력
            time.sleep(2)  # 잠시 대기 후 재시도

    else:  # for 루프를 break로 빠져나오지 못한 경우 (= 세 번 모두 실패한 경우)
        # 3번 시도 후에도 실패하면 2(판단불가)로 기록하고 넘어감
        df_sample_pred.at[idx, "llm_contro_label"] = 2                  # 라벨을 2(판단불가)로 저장
        df_sample_pred.at[idx, "llm_contro_raw"]   = f"ERROR: {last_error}"  # 에러 메시지를 원문 응답 컬럼에 기록
        continue  # 다음 행으로 넘어감

    content = resp.choices[0].message.content  # LLM이 생성한 메시지 내용에 접근
    if content is None:                        # content가 None일 경우 방어 코드
        content = ""
    content = str(content).strip()             # 문자열로 변환 후 앞뒤 공백 제거

    # 기본값은 2(판단불가), 응답 안에서 0/1/2 중 첫 등장 숫자를 라벨로 사용
    label = 2  # 초기값은 2 (모호/판단불가)
    for ch in content:              # LLM 응답 문자열을 앞에서부터 한 글자씩 확인
        if ch in ("0", "1", "2"):   # 0, 1, 2 중 하나가 처음 등장하는 순간
            label = int(ch)         # 그 숫자를 최종 라벨로 채택
            break                   # 루프 종료

    df_sample_pred.at[idx, "llm_contro_label"] = label    # 추출된 라벨(0/1/2)을 DataFrame에 저장
    df_sample_pred.at[idx, "llm_contro_raw"]   = content  # LLM이 반환한 전체 응답 텍스트를 저장 (디버깅/후분석용)

    # 안전하게 중간 저장 (예: 1000건마다)
    if idx % 1000 == 0 and idx > 0:  # 인덱스가 1000의 배수이고 0보다 큰 경우에만 중간 저장
        df_sample_pred.to_csv(       # 현재까지의 예측 결과를 임시 CSV로 저장
            "/home/work/CSG/snv/dc_sample_50k_with_llm_tmp.csv",
            index=False,
            encoding="utf-8-sig",    # 한글 호환을 위해 utf-8-sig 인코딩 사용
        )


In [7]:
df_sample_pred['llm_contro_label'].value_counts()

llm_contro_label
0    18269
1     1731
Name: count, dtype: int64

In [135]:
# 최종 저장
df_sample_pred.to_csv(
    "/home/work/CSG/snv/dc_sample_20k_with_llm_labels.csv",
    index=False,
    encoding="utf-8-sig",
)

In [139]:
# KOTE에서 제시한 44개 감정 라벨

LABELS = ['불평/불만', '환영/호의', '감동/감탄', '지긋지긋', '고마움', '슬픔', '화남/분노', '존경', '기대감', '우쭐댐/무시함', '안타까움/실망', '비장함', '의심/불신', '뿌듯함', '편안/쾌적',
          '신기함/관심', '아껴주는', '부끄러움', '공포/무서움', '절망', '한심함', '역겨움/징그러움', '짜증', '어이없음', '없음', '패배/자기혐오', '귀찮음', '힘듦/지침', '즐거움/신남',
          '깨달음', '죄책감', '증오/혐오', '흐뭇함(귀여움/예쁨)', '당황/난처', '경악', '부담/안_내킴', '서러움', '재미없음', '불쌍함/연민', '놀람', '행복', '불안/걱정', '기쁨', '안심/신뢰']

device = "cuda" if torch.cuda.is_available() else "cpu" # CUDA 사용 가능 여부에 따라 실행 장치 설정

class KOTEtagger(pl.LightningModule):  # 감정 분류 모델 클래스를 정의
    def __init__(self):  # 클래스가 생성될 때 한 번 실행되는 초기화 함수
        super().__init__()  # LightningModule의 초기화 내용도 같이 실행
        self.electra = ElectraModel.from_pretrained("beomi/KcELECTRA-base", revision='v2021').to(device)  # 텍스트 의미를 잘 담은 토큰별 은닉벡터를 얻기 위해 한국어에 맞게 사전학습된 ELECTRA 모델 불러와 지정된 디바이스(GPU/CPU)로 이동
        self.tokenizer = AutoTokenizer.from_pretrained("beomi/KcELECTRA-base", revision='v2021')  # 위 ELECTRA 모델과 호환되는 토크나이저 로드 (문자열 → 토큰 ID 변환 담당)
        self.classifier = nn.Linear(self.electra.config.hidden_size, 44).to(device)  # ELECTRA의 [CLS] 임베딩 차원(히든 크기)을 44개 감정 라벨 확률로 변환하는 선형층 정의

    def forward(self, text: str):  # 모델을 호출했을 때 실행되는 함수; 한 개의 문자열 단위로 입력됨
        encoding = self.tokenizer.encode_plus(  # 토크나이저로 텍스트를 숫자 ID로 변환하고, 패딩/마스크까지 한 번에 준비
          text,  # 분류할 원문 문자열
          add_special_tokens=True,  # 문장 앞뒤에 [CLS], [SEP] 같은 특수 토큰 자동 추가 (모델이 문장 경계 인식 가능)
          max_length=512,  # 토큰 최대 길이 제한 (너무 긴 문장은 512 토큰으로 잘림; 너무 짧은 문장은 패딩으로 채움)
          return_token_type_ids=False,  # 문장쌍 분류가 아니라서 세그먼트 ID는 사용하지 않음
          padding="max_length",  # 길이가 512가 되도록 패딩 (배치 처리 시 텐서 크기를 맞추기 위함)
          return_attention_mask=True,  # 실제 토큰과 패딩을 구분해주는 마스크도 반환 (모델이 패딩을 무시하도록)
          return_tensors='pt',  # 파이토치 텐서 형태로 반환 (바로 모델에 넣을 수 있음)
        ).to(device)  # 생성된 텐서들(input_ids, attention_mask)을 GPU/CPU로 이동

        output = self.electra(  # 사전학습 모델에 인풋 전달하여 마지막 히든 스테이트(각 토큰 임베딩) 얻기
            encoding["input_ids"],  # 토큰 ID 텐서
            attention_mask=encoding["attention_mask"]  # 패딩 위치를 0으로 하여 모델이 무시하게 함
        )
        output = output.last_hidden_state[:, 0, :]  # 배치의 각 문장에서 첫 번째 토큰([CLS]) 위치의 임베딩만 추출 (문장 대표 벡터)
        output = self.classifier(output)  # [CLS] 임베딩을 선형층에 통과시켜 44차원 로짓(각 감정 라벨의 점수)으로 변환
        output = torch.sigmoid(output)  # 다중 레이블 분류이므로 각 라벨에 대해 독립 확률(0~1)로 해석하기 위해 시그모이드 적용
        torch.cuda.empty_cache()  # (선택) GPU 메모리 캐시를 비워 메모리 부족 문제를 완화 (빈번 호출은 성능에 영향 가능)

        return output  # 크기: (배치크기, 44) 의 텐서 반환. 한 문장만 넣었으니 (1, 44). 각 값은 해당 감정 라벨의 확률

# 학습된 가중치를 불러와 즉시 예측 가능한 모델 객체 생성
trained_model = KOTEtagger()  # 위에서 정의한 모델 클래스 인스턴스화 (백본+분류기 구조만 포함, 아직 가중치는 랜덤)
trained_model.load_state_dict(  # 학습이 완료된 파라미터(가중치)를 파일에서 읽어와 모델에 로드
    torch.load("/home/work/CSG/snv/kote_pytorch_lightning.bin"),  # 구글 드라이브에 저장된 체크포인트 경로
    strict=False  # 레이어 이름/개수가 조금 달라도 가능한 항목만 로드(미스매치가 있어도 최대한 불러오도록)
)  # 이 시점부터 trained_model은 학습된 상태로 텍스트 입력 시 바로 감정 확률을 출력할 수 있음


_IncompatibleKeys(missing_keys=[], unexpected_keys=['electra.embeddings.position_ids'])

In [140]:
tmp = df_cases.copy()                                            # df_cases를 복사하여 원본 훼손 없이 전처리용 임시 DataFrame 생성
tmp["artist_norm"] = tmp["artist"].str.strip().str.casefold()    # 아티스트명을 공백 제거 후 소문자/케이스 표준화(casefold)하여 비교용 정규화 컬럼 추가

right = (tmp[cols + ["artist_norm"]]                             # 원하는 컬럼들(cols)과 정규화된 artist_norm 컬럼만 선택
         .drop_duplicates(subset="artist_norm", keep="first")    # artist_norm 기준으로 중복된 행 제거(첫 번째 행만 유지)
         .drop(columns="artist_norm"))                           # 정규화 컬럼은 이후 불필요하므로 제거하여 최종 테이블 구성

right                                                              # 정제된 아티스트 메타 정보 DataFrame 확인


,artist,group,sex_target,acttype,listorg,type
0,(여자)아이들 ((G)I-DLE),1,2,2,1,1
11,2NE1,3,2,2,1,1
18,2am,4,1,1,1,1
21,B.A.P,2,1,1,0,1
24,DKZ,1,1,1,0,1
...,...,...,...,...,...,...
900,포미닛(4Minute),3,2,2,1,1
915,프로미스나인 (fromis_9),1,2,2,0,5
919,프리스틴 (PRISTIN),2,2,2,0,1
922,피원하모니(P1Harmony),1,1,1,1,1


In [153]:
# df_sample_pred 에 df_cases 정보 조인
out = df_sample_pred.merge(right, on="artist", how="left")
print(len(out))
out.head()

20000


,artist,content,ip,like,month,nickname,title,uid,unlike,url,...,논란확률_KcELECTRA,class,token_count,llm_contro_label,llm_contro_raw,group,sex_target,acttype,listorg,type
0,JYJ,도키도키!!!!!!!! 내일이면 움직이는 티저가.........!!!!!!!,211.36,0,2015-02,레오뿌냐,현재 이 시각 꽃 발매와 꽃콘까지 남은 시간...!,NaN,0.0,https://gall.dcinside.com/board/view/?id=jyj&n...,...,0.097260,NaN,58,2,,3.0,1.0,1.0,0.0,4.0
1,마마무 (Mamamoo),[1위 예측] 옥탑방 (-0.09285) 4.82998|5.23084|4.9508...,NaN,0,2019-03,맘갤차트봇,[14:15] 멜론 5분 차트,mgchartbot2,0.0,https://gall.dcinside.com/board/view/?id=mamam...,...,0.092829,NaN,154,2,,2.0,2.0,2.0,1.0,1.0
2,엑소 (EXO),ㅋ,175.223,3,2017-12,ㅇㅇ,뿡 머상주기 싫어서 가온이 기사냄ㅋ,NaN,0,https://gall.dcinside.com/board/view/?id=exo_n...,...,0.135178,NaN,29,2,,2.0,1.0,1.0,1.0,1.0
3,엑소 (EXO),ㅈㄴ 싫어,210.104,0,2019-01,ㅇㅇ,혜나 왜저러냐 진짜,NaN,0,https://gall.dcinside.com/board/view/?id=exo_n...,...,0.341638,NaN,26,2,,2.0,1.0,1.0,1.0,1.0
4,마마무 (Mamamoo),에어팟 1vs2,118.40,0,2019-03,ㅇㅇ,ㅃ vs,NaN,0,https://gall.dcinside.com/board/view/?id=mamam...,...,0.095476,NaN,13,2,,2.0,2.0,2.0,1.0,1.0


In [154]:
import statsmodels.formula.api as smf  # 회귀모형 등 통계 모델링을 위한 statsmodels formula API
from sklearn.metrics import mean_absolute_error, r2_score  # MAE, R² 계산을 위한 평가 지표 함수

out['sex_target'] = out['sex_target'].replace({1: 0, 2: 1})  # 성별 타깃 변수를 1→0, 2→1로 재코딩 (예: 0=남, 1=여 로 통일)

out['anonym'] = (~(out['uid'].notna() & out['uid'].astype(str).str.strip().ne(''))).astype(int)  
# uid가 비어있거나 공백이면 1(익명), 값이 있으면 0(비익명)으로 anonym 변수 생성
# - out['uid'].notna() : NaN이 아닌지 여부
# - .astype(str).str.strip().ne('') : 공백이 아닌 문자열인지 여부
# - 둘 다 만족하는 경우를 뒤집어서(~) 익명 여부로 사용

use_cols = ["sex_target", "group", "anonym", "texts", "논란여부_KcELECTRA"]  # 분석에 사용할 컬럼 목록 정의
df_model = out[use_cols]  # 위 컬럼만 추출해 모델용 DataFrame 생성

# out = out.dropna()
df_model.reset_index(drop=True, inplace=True)  # 인덱스를 0부터 다시 부여하고 기존 인덱스는 버림

df_model = df_model[df_model['논란여부_KcELECTRA'] == 1]  # KcELECTRA가 1(논란 관련 글)로 분류한 행만 필터링

print(len(df_model))  # 최종 필터링 후 남은 행(논란 관련 글)의 개수 출력


1902


In [155]:
# 각 문장의 토큰 길이 계산 함수
def token_len(text):
    if text is None:
        text = "" # 결측값 방지: None이면 빈 문자열로 처리(특수 토큰 길이만 남게 됨)
    enc = tokenizer.encode_plus(
        str(text),
        add_special_tokens=True, # [CLS], [SEP] 등 특수 토큰 포함 -> 길이에도 포함됨
        padding=False, # 패딩 미적용(실제 토큰화 결과의 원래 길이 측정)
        truncation=False, # 자르지 않음(길이를 정확히 알고 싶기 때문)
        return_tensors="pt"
    )
    return enc["input_ids"].shape[1]

# 길이 컬럼 추가
df_model["seq_len"] = df_model["texts"].apply(token_len)

# 512 초과 제거(ELECTRA/BERT류 최대 입력 길이를 넘는 샘플 필터링)
df_model = df_model[df_model["seq_len"] <= 512].copy() # 위에서 max_length 가 512로 설정되어 있으므로 이를 초과하는 문서는 오류가 발생함(물론 파라미터로 처리는 가능하나 여기서는 지나치게 긴 문서를 제거하는 방향으로 진행)
df_model.reset_index(drop=True, inplace=True)
print(len(df_model))

Token indices sequence length is longer than the specified maximum sequence length for this model (574 > 512). Running this sequence through the model will result in indexing errors


1894


In [156]:
neg_p = []  # 부정 감정 평균 점수 저장 리스트
pos_p = []  # 긍정 감정 평균 점수 저장 리스트
mean_p = []  # 전체 감정 점수 평균 저장 리스트
labels = []  # 감지된 감정 레이블 저장 리스트
values = []  # 감정 점수 저장 리스트

# 감정 라벨 분류: 긍정/부정/중립
pos = ['환영/호의', '감동/감탄', '고마움', '존경', '기대감', '뿌듯함', '편안/쾌적', '신기함/관심', '아껴주는', '즐거움/신남', '흐뭇함(귀여움/예쁨)', '행복', '기쁨', '안심/신뢰']
neg = ['불평/불만', '지긋지긋', '슬픔', '화남/분노', '안타까움/실망', '의심/불신', '부끄러움', '공포/무서움', '절망', '한심함', '역겨움/징그러움', '짜증', '어이없음',
       '패배/자기혐오', '귀찮음', '힘듦/지침', '죄책감', '증오/혐오', '당황/난처', '경악', '부담/안_내킴', '서러움', '재미없음', '불쌍함/연민', '불안/걱정']
neu = ['우쭐댐/무시함', '비장함', '없음', '깨달음', '놀람']

# 전체 텍스트에 대해 감정 추론 수행
for text in tqdm(df_model['texts']):
    preds = trained_model(text)[0]  # 감정 예측 확률 반환
    test = []  # 감지된 감정 레이블 저장
    test1 = []  # 해당 감정의 예측 점수 저장
    test2 = []  # 긍/부/중립에 따라 점수 조정하여 저장

    # 예측된 감정 중 threshold(0.4) 이상인 항목만 저장
    for label, value in zip(LABELS, preds):
        if value>0.4:
            test.append(label)
            test1.append(round(value.item(), 2))
    labels.append(test)

    # 감정 라벨 분류별 점수 부여: 부정은 음수, 긍정은 양수, 중립은 0
    for l, s in zip(test, test1):
        if l in neg:
            test2.append(s * -1)
        elif l in neu:
            test2.append(0)
        elif l in pos:
            test2.append(s)
    values.append(test2)

    # 긍정 및 부정 점수 분리
    positive_numbers = []
    negative_numbers = []

    for num in test2:
        if num > 0:
            positive_numbers.append(num)
        elif num < 0:
            negative_numbers.append(num)

    # 각 감정 점수 평균값 계산 (감정 없을 경우 0 처리)
    try:
        positive = round(sum(positive_numbers) / len(positive_numbers), 3)
        pos_p.append(positive)
    except:
        positive = 0
        pos_p.append(positive)
    try:
        negative = round(sum(negative_numbers) / len(negative_numbers), 3)
        neg_p.append(negative)
    except:
        negative = 0
        neg_p.append(negative)

    mean_p.append(round(positive + negative, 3))

100% 1894/1894 [04:53<00:00,  6.44it/s]


In [157]:
df_model['sentiment_label'] = [', '.join(map(str, lst)) for lst in labels]
df_model['sentiment_values'] = [', '.join(map(str, lst)) for lst in values]
df_model['sentiment_pos'] = pos_p
df_model['sentiment_neg'] = neg_p
df_model['sentiment_mean'] = mean_p

In [161]:
formula = "sentiment_mean ~ sex_target + C(group) + anonym"
model   = smf.ols(formula, data=df_model).fit(cov_type="HC3")
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:         sentiment_mean   R-squared:                       0.028
Model:                            OLS   Adj. R-squared:                  0.025
Method:                 Least Squares   F-statistic:                     9.470
Date:                Fri, 21 Nov 2025   Prob (F-statistic):           6.37e-09
Time:                        03:01:19   Log-Likelihood:                -1077.6
No. Observations:                1739   AIC:                             2167.
Df Residuals:                    1733   BIC:                             2200.
Df Model:                           5                                         
Covariance Type:                  HC3                                         
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept          -0.4286      0.041    -